<a href="https://colab.research.google.com/github/andryll/MGR-IC/blob/main/codes/Feature_Extraction_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip uninstall h5py


^C
Note: you may need to restart the kernel to use updated packages.


<a href="https://colab.research.google.com/github/andryll/MGR-IC/blob/main/codes/Feature_Extraction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as Bibliotecas

import numpy as np
import librosa
#import IPython.display as ipd
import os
import pandas as pd

import librosa.display
import matplotlib.pyplot as plt
import io
from PIL import Image
from keras.utils import to_categorical
import random


In [ ]:
def splitSongs (songList, duration, sr=44100):

  # Converta o tamanho da janela de segundos para amostras
  window_size_samples = int(duration * sr)

  # Inicialize uma lista para armazenar os segmentos
  segmentedList = []

  # Pega cada música da lista
  for y in songList:

    # Calcule o número total de segmentos
    num_segments = len(y[0]) // window_size_samples

    # Divida o áudio em segmentos de 5 segundos e adcione-os na lista
    for i in range(num_segments):
        start = i * window_size_samples
        end = (i + 1) * window_size_samples
        segment = (y[0][start:end], y[1], y[2])
        segmentedList.append(segment)

  # Retorna a nova lista
  return segmentedList

In [ ]:
def readSongs (genre, numSongs, sr=44100, duration = 30):

  # Declarando listas iniciais
  songs = []
  genrelist = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
                'metal', 'pop', 'reggae', 'rock']

  # Se a escolha de gênero não for 'all', substitui a lista pelo gênero escolhido
  if genre != 'all':
    genrelist = [genre]

  #Percorre todos os gêneros da lista
  for g in genrelist:
    #Pega o caminho para a pasta do gênero escolhido
    dir_path = os.path.join('C:/Users/andry/OneDrive/Documentos/GitHub/MGR-IC/songs', g)
    #Lista os arquivos da pasta e os embaralha
    files = os.listdir(dir_path)
    files.sort

    # Até o número de musicas desejado ser alcançado, lê os arquivos de áudio com o librosa
    for i in range(numSongs):
      songs.append(librosa.load(os.path.join(dir_path, files[i]), sr=sr, mono = True, duration = 30))
      # Adciona o gênero como uma variável da tupla
      songs[-1] = songs[-1] + (g,)

  max_len = max(len(song[0]) for song in songs)

  # Garante que todas as músicas terão o mesmo tamanho da maior
  resized_songs = []
  for song in songs:
      # Verifica se a música precisa ser redimensionada
      if len(song[0]) < max_len:
          # Adiciona zeros à direita para igualar o tamanho
          padded_audio = librosa.util.pad_center(data = song[0], size = max_len, axis = 0)
          resized_songs.append((padded_audio, song[1], song[2]))
      else:
          resized_songs.append(song)

  new_songs = splitSongs (resized_songs, sr=sr, duration = duration)

  return new_songs

### Extração das Features


In [ ]:
def featureExtraction (songs, sr=44100):

  colunas = ['spectogram', 'Classe']

  df = pd.DataFrame(columns=colunas)

  for y in songs:

    S = librosa.feature.melspectrogram(y=y[0], sr=y[1])
    S_dB = librosa.power_to_db(S, ref=np.max)

    # Criar uma figura do matplotlib para o espectrograma
    dpi=100
    size = 254
    figsize = (size/dpi, size/dpi)
    plt.figure(figsize=figsize, dpi=dpi)
    librosa.display.specshow(S_dB, sr=y[1])
    plt.tight_layout()

    # Salvar o espectrograma em um buffer de memória
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()
    buf.seek(0)

    img = Image.open(buf).convert("RGB")
    img_array = np.array(img)/255

    #Adcionando ao DF

    df.loc[len(df)] = [img_array, y[2]]

  return df

### Execução

In [ ]:
# Lê as músicas
songs = readSongs ('all', 10, 44100, 5)


In [ ]:
len(songs)

600

In [ ]:
songs[0][0]

y= songs[0]

S = librosa.feature.melspectrogram(y=y[0], sr=y[1])
S_dB = librosa.power_to_db(S, ref=np.max)

# Criar uma figura do matplotlib para o espectrograma
dpi=100
size = 254
figsize = (size/dpi, size/dpi)
plt.figure(figsize=figsize, dpi=dpi)
librosa.display.specshow(S_dB, sr=y[1])
plt.tight_layout()

# Salvar o espectrograma em um buffer de memória
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
plt.close()
buf.seek(0)

img = Image.open(buf).convert("RGB")
img_array = np.array(img)/255

In [ ]:
df = featureExtraction (songs, sr=44100)

In [ ]:
df['Classe'] = pd.factorize(df['Classe'])
display(df)

,spectogram,Classe
0,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
3,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
4,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
...,...,...
595,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",9
596,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",9
597,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",9
598,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",9


In [ ]:
csvnames = ['C:/Users/andry/OneDrive/Documentos/GitHub/MGR-IC/ft2/5s/spec.csv']

df.to_csv(csvnames[0], sep=',', index=False, encoding='utf-8')